In [ ]:
import sys
sys.path.append("../")

# Get rid of annoying tf warning
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import bayesflow as beef
import pandas as pd

from experiments import RandomWalkDiffusionExperiment
from models import RandomWalkDiffusion

from priors import sample_scale, sample_random_walk, sample_ddm_params
from likelihoods import sample_static_diffusion_process, _sample_diffusion_trial, sample_random_walk_diffusion_process

In [ ]:
# If set to False, existing results will be loaded
# Set to True if you want to re-run the experiments
TRAIN_NETWORKS = False

## Neural Experiment

In [ ]:
model = RandomWalkDiffusion()

In [ ]:
neural_experiment = RandomWalkDiffusionExperiment(model)

### Training

In [ ]:
if TRAIN_NETWORKS:
    history = neural_experiment.run(
        epochs=100, 
        iterations_per_epoch=1000, 
        batch_size=8
    )
else:
    history = neural_experiment.trainer.loss_history.get_plottable()

In [ ]:
f = beef.diagnostics.plot_losses(history)

### Evaluation

In [ ]:
# prepare data
data = pd.read_csv('../data/optimal_policy_data.csv')

In [ ]:
which = 0
person_data = {"summary_conditions": data['rt'].loc[data['id'] == which + 1].to_numpy()[None, ..., None]}

In [ ]:
posterior_samples = neural_experiment.amortizer.sample(person_data, 2000)